In [1]:
import geopandas as gpd
import pandas as pd

# Load data

In [2]:
parcels = gpd.read_file('la_parcels/Parcels.shp')

In [3]:
parcels.shape

(881522, 18)

In [4]:
# sites_df = pd.read_excel('Appendix 4.7 Candidate Sites For Rezoning-Table B.xlsx', skiprows=1)

In [4]:
sites_df = pd.read_parquet('sites.parquet')

In [5]:
sites_df['PIN'].isin(parcels['PIN']).mean()

0.9990280410166691

In [6]:
(~sites_df['PIN'].isin(parcels['PIN'])).sum()

260

In [8]:
pd.set_option('max_columns', 50)

In [9]:
sites_df

,Jurisdiction Name,Site Address/Intersection,5 Digit ZIP Code,Assessor Parcel Number,Very Low-Income,Low-Income,Moderate-Income,Above Moderate-Income,Type of Shortfall\n,Parcel Size\n(Acres),Current General Plan Designation,Current Zoning,Proposed General Plan (GP) Designation,Proposed Zoning,Minimum Density Allowed,Maximum Density Allowed,Total Capacity,Vacant/\nNonvacant,Description of Existing Uses,PIN,Base Density Per Acre,CPA_Name,RSO,Applicable Rezoning Programs\n(Program informing values in column Q listed first)
0,LOS ANGELES,366 CITRUS AVE,90036.0,5.524038e+09,NaN,NaN,0.085,NaN,Shortfall of Sites,0.201446,Low II Residential,R1-1,None,None,NaN,9,0.085,None,"Residential - Single Family Residence, 1",138B185 19,nan,Wilshire,None,ADU
1,LOS ANGELES,227 WILTON PL,90004.0,5.516010e+09,NaN,NaN,0.085,NaN,Shortfall of Sites,0.234353,Low II Residential,R1-1,None,None,NaN,8,0.085,None,"Residential - Single Family Residence, 1",135B189 25,nan,Wilshire,None,ADU
2,LOS ANGELES,239 WILTON PL,90004.0,5.516010e+09,NaN,NaN,0.085,NaN,Shortfall of Sites,0.234343,Low II Residential,R1-1,None,None,NaN,8,0.085,None,"Residential - Single Family Residence, 1",135B189 66,nan,Wilshire,None,ADU
3,LOS ANGELES,245 WILTON PL,90004.0,5.516010e+09,NaN,NaN,0.085,NaN,Shortfall of Sites,0.234340,Low II Residential,R1-1,None,None,NaN,8,0.085,None,Residential - Single Family Residence - Mills ...,135B189 87,nan,Wilshire,None,ADU
4,LOS ANGELES,300 CITRUS AVE,90036.0,5.524038e+09,NaN,NaN,0.085,NaN,Shortfall of Sites,0.204095,Low II Residential,R1-1,None,None,NaN,9,0.085,None,"Residential - Single Family Residence, 1",138B185 103,nan,Wilshire,None,ADU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267496,LOS ANGELES,7531 NAYLOR AVE,90045.0,4.106002e+09,NaN,NaN,NaN,6.12,Shortfall of Sites,0.084339,General Commercial,[Q]C2-1,None,None,NaN,108,6.120,None,"Commercial - Store - One Story, 0",102B165 881,43,Westchester - Playa del Rey,None,WPDR CPU
267497,LOS ANGELES,7531 NAYLOR AVE,90045.0,4.106002e+09,NaN,NaN,NaN,5.44,Shortfall of Sites,0.082489,General Commercial,[Q]C2-1,None,None,NaN,108,5.440,None,"Commercial - Store - One Story, 0",102B165 900,43,Westchester - Playa del Rey,None,WPDR CPU
267498,LOS ANGELES,0,0.0,4.104006e+09,NaN,NaN,NaN,4.80,Shortfall of Sites,0.046324,Public Facilities,PF-1,None,None,NaN,108,4.800,Vacant,Residential - Single Family Residence - Vacant...,099B169 35,43,Westchester - Playa del Rey,None,WPDR CPU
267499,LOS ANGELES,7616 SEPULVEDA BLVD,90045.0,4.106018e+09,NaN,NaN,NaN,3.60,Shortfall of Sites,0.051494,General Commercial,[Q]C2-1,None,None,NaN,108,3.600,None,"Commercial - Store - One Story, 1",099B165 80,43,Westchester - Playa del Rey,None,WPDR CPU


# Extract existing number of units

In [17]:
sites_df['existing_units'] = sites_df['Description of Existing Uses'].str.rsplit(',').str[-1]

# Merge data

In [18]:
merged_df = parcels[['PIN', 'geometry']].merge(
    sites_df,
    how='right',
    on='PIN'
)

# Save merged data

In [25]:
merged_df_crs = merged_df.to_crs('EPSG:4326')

merged_df_crs['density'] = pd.to_numeric(merged_df_crs['Maximum Density Allowed'], errors='coerce')

merged_df_crs['no_max_density'] = (merged_df_crs['Maximum Density Allowed'] == 'No Max')

merged_df_crs['units'] = (
    merged_df_crs['density'] * merged_df_crs['Parcel Size\n(Acres)']
).round(1)

merged_df_crs['realistic_capacity'] = merged_df_crs['Total Capacity']

merged_df_crs['zoning'] = merged_df_crs['Current Zoning']

merged_df_crs['address'] = merged_df_crs['Site Address/Intersection']

In [30]:
(merged_df_crs['address'] == '0 ').sum()

8400

In [26]:
merged_df_crs.geometry.total_bounds

array([-118.66755417,   33.7064461 , -118.1555711 ,   34.32457583])

In [27]:
merged_df_crs[[
    'geometry', 'zoning', 'units', 'density', 'realistic_capacity', 'no_max_density', 'existing_units', 
    'address', 'PIN'
]].dropna(subset=['geometry']).to_file('out.geojson', driver='GeoJSON')

In [28]:
points_df = merged_df_crs.copy()
points_df.geometry = points_df.geometry.centroid

points_df[[
    'geometry', 'zoning', 'units', 'density', 'realistic_capacity', 'no_max_density', 'existing_units', 
    'address', 'PIN'
]].dropna(subset=['geometry']).to_file('points.geojson', driver='GeoJSON')

/var/folders/y6/q_9gyvlj7ln20t8tgvcpmfcr0000gn/T/ipykernel_22799/1185456878.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  points_df.geometry = points_df.geometry.centroid
